In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy
from collections import Counter

데이터 가져오기

In [51]:
c_score = pd.read_csv('c_score_rate.csv')
vod_list = pd.read_csv('vod_add_id.csv', index_col=0)

In [52]:
c_score

,subsr,rename,Category,vod_id,Affinity,ratings
0,59879000,나쁜엄마,TV프로그램,3403,0.405465,1.0
1,59879000,나의해방일지,TV프로그램,1859,0.405465,1.0
2,59879000,낭만닥터김사부2,TV프로그램,1168,0.405465,1.0
3,59879000,소방서옆경찰서,TV프로그램,2092,1.504077,3.0
4,59879000,슬기로운의사생활시즌2,TV프로그램,2891,0.405465,1.0
...,...,...,...,...,...,...
7700,67160000,심야괴담회,TV프로그램,143,0.637052,1.0
7701,67160000,콘크리트유토피아,영화,2630,0.659090,3.0
7702,67161000,스파이코드명포춘,영화,3824,1.705884,5.0
7703,67164000,미션임파서블:데드레코닝PARTONE,영화,2867,1.083268,1.0


In [53]:
# BigCategory, SmallCategory로 장르 생성
vod_list['genres'] = vod_list['BigCategory'].str.replace('/', '').str.replace(' ', '') + ' ' + vod_list['SmallCategory'].str.replace('/', '').str.replace(' ', '')
vod_list

,name,Summary,actors,director,BigCategory,SmallCategory,Category,rename,counts,vod_name,running_time,vod_id,genres
0,와이낫크루 뜻밖의 여행,가는 길이 좀 멀고 험해도 나만의 취저(취향 저격)인 곳은 집요하게 찾아가는 MZ를...,"KCM,권은비,조정식,박현규","은지향, 김한진, 김남훈, 명민아, 김주희",TV 연예/오락,기타,TV프로그램,와이낫크루뜻밖의여행,16,와이낫크루: 뜻밖의 여행,69분,0,TV연예오락 기타
1,그레이트 뷰티,40여 년 전 소설 한 권을 끝으로 더 이상 책을 쓰지 못하는 젭은 로마 1%의 삶...,"토니 세르빌로,사브리나 페릴리,세레나 그랜디,이사벨라 페라리",파올로 소렌티노,영화,기타,영화,그레이트뷰티,1,그레이트 뷰티,141분,1,영화 기타
2,해안선,평온해 보이는 동해안의 바닷가. “경고! 밤 7시 이후 이곳을 접근하는 자는 간첩으...,"장동건,김정학,박지아,유해진",김기덕,영화,액션/어드벤쳐,영화,해안선,1,해안선,94분,2,영화 액션어드벤쳐
3,아는 형님,이성 상실 본능 충실 형님학교에서 벌어지는 세상의 모든 놀이 [아는 형님],"강호동,서장훈,김영철,이수근,김희철,민경훈,이상민","민철기, 최창수, 이수아, 황선영, 지은정",TV 연예/오락,기타,TV프로그램,아는형님,44,아는 형님,87분,3,TV연예오락 기타
4,베놈,진실을 위해서라면 몸을 사리지 않는 정의로운 열혈 기자 '에디 브록' 거대 기업 라...,"톰 하디,미셀 윌리엄스,리즈 아메드,제니 슬레이트,레이드 스콧,스콧 헤이즈,우디 해...",루벤 플레셔,영화,액션/어드벤쳐,영화,베놈,2,베놈,107분,4,영화 액션어드벤쳐
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4710,베놈 2: 렛 데어 비 카니지,‘베놈’과 완벽한 파트너가 된 ‘에디 브록’(톰 하디) 앞에 ‘클리터스 캐서디’(우...,"톰 하디,미셀 윌리엄스,나오미 해리스,레이드 스콧,스티븐 그레이엄,우디 해럴슨",앤디 서키스,영화,액션/어드벤쳐,영화,베놈2:렛데어비카니지,11,베놈 2: 렛 데어 비 카니지,97분,4710,영화 액션어드벤쳐
4711,성월동화,야쿠자 두목 사토는 대도 당걸에게 자신의 일기를 찾도록 의뢰한다. 그러나 당걸에게 ...,"장국영,토키와 타카코,양자경,이찬삼,고첩,혜천사,요계지,황가락",진가상,영화,멜로,영화,성월동화,3,성월동화 2 - 연전충승,98분,4711,영화 멜로
4712,다큐인사이트 코드블루,히포크라테스 선서를 외치고 의사들이 출발선에 서는 3월. 전남대병원에 발 디딘 새내...,유승호,-,TV 시사/교양,기타,TV프로그램,다큐인사이트코드블루,1,다큐인사이트 코드블루,47분,4712,TV시사교양 기타
4713,캠프 제로베이스원,9인만의 스페셜 에디션 캠핑카를 타고 떠나는 예능 프로그램,제로베이스원,서예진,TV 연예/오락,기타,TV프로그램,캠프제로베이스원,1,캠프 제로베이스원,109분,4713,TV연예오락 기타


In [54]:
# 장르 정보 추출
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer()
genres = cv.fit_transform(vod_list.genres)

cv.vocabulary_

{'tv연예오락': 5,
 '기타': 8,
 '영화': 29,
 '액션어드벤쳐': 26,
 '코미디': 38,
 'tv시사교양': 3,
 'tv드라마': 2,
 '외화시리즈': 31,
 '키즈': 39,
 '오락': 30,
 '애니메이션': 24,
 'tv애니메이션': 4,
 '명랑코믹': 18,
 '미분류': 22,
 '인물다큐': 35,
 '멜로': 17,
 '학원순정연애': 41,
 '드라마': 13,
 'sf메카': 0,
 '라이프': 14,
 '운동건강': 33,
 '공포스릴러': 7,
 '스포츠': 23,
 '레슬링': 15,
 '다큐': 10,
 '인물': 34,
 '역사': 27,
 '다큐멘터리': 11,
 'sf환타지': 1,
 '단편': 12,
 '액션모험': 25,
 '로맨틱코미디': 16,
 '자연': 36,
 '학습': 40,
 '무협': 19,
 '추리미스터리': 37,
 '무협환타지': 20,
 '공연음악': 6,
 '뮤지컬': 21,
 '우리동네': 32,
 '연예오락': 28,
 '뉴스': 9}

In [55]:
# one-hot vector 생성
genres = pd.DataFrame(genres.toarray(), columns=list(sorted(cv.vocabulary_.keys(), key=lambda x:cv.vocabulary_[x])))
genres

,sf메카,sf환타지,tv드라마,tv시사교양,tv애니메이션,tv연예오락,공연음악,공포스릴러,기타,뉴스,...,우리동네,운동건강,인물,인물다큐,자연,추리미스터리,코미디,키즈,학습,학원순정연애
0,0,0,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4710,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4711,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4712,0,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4713,0,0,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


각 contents 간 거리 계산

In [94]:
from sklearn.neighbors import NearestNeighbors

# n_neighbors: 가장 가까운 n개의 이웃을 찾도록 지정합니다.
nbrs = NearestNeighbors(n_neighbors=20).fit(genres)
nbrs

NearestNeighbors(n_neighbors=20)

In [95]:
# 0번째 vod와 1번째 vod의 유클리디언 거리를 계산하기
a = genres.iloc[0, :]
print(a)
b = genres.iloc[1, :]
print(b)
np.linalg.norm(a-b)

sf메카       0
sf환타지      0
tv드라마      0
tv시사교양     0
tv애니메이션    0
tv연예오락     1
공연음악       0
공포스릴러      0
기타         1
뉴스         0
다큐         0
다큐멘터리      0
단편         0
드라마        0
라이프        0
레슬링        0
로맨틱코미디     0
멜로         0
명랑코믹       0
무협         0
무협환타지      0
뮤지컬        0
미분류        0
스포츠        0
애니메이션      0
액션모험       0
액션어드벤쳐     0
역사         0
연예오락       0
영화         0
오락         0
외화시리즈      0
우리동네       0
운동건강       0
인물         0
인물다큐       0
자연         0
추리미스터리     0
코미디        0
키즈         0
학습         0
학원순정연애     0
Name: 0, dtype: int64
sf메카       0
sf환타지      0
tv드라마      0
tv시사교양     0
tv애니메이션    0
tv연예오락     0
공연음악       0
공포스릴러      0
기타         1
뉴스         0
다큐         0
다큐멘터리      0
단편         0
드라마        0
라이프        0
레슬링        0
로맨틱코미디     0
멜로         0
명랑코믹       0
무협         0
무협환타지      0
뮤지컬        0
미분류        0
스포츠        0
애니메이션      0
액션모험       0
액션어드벤쳐     0
역사         0
연예오락       0
영화         1
오락         0
외화시리즈      0
우리동네       0
운동건

1.4142135623730951

In [96]:
# 0번째 vod와 가까운 10개 vod
distances, indices = nbrs.kneighbors([a])
distances
indices
recommendations = vod_list.loc[indices[0], ["vod_id"]]
recommendations
recommendations["distance"] = distances[0]
recommendations

C:\Users\USER\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


,vod_id,distance
1387,1387,0.0
4519,4519,0.0
292,292,0.0
3838,3838,0.0
4082,4082,0.0
271,271,0.0
2919,2919,0.0
4092,4092,0.0
290,290,0.0
280,280,0.0


장르 유사한 10개에 평점 정보 결합

In [97]:
# vod_id 별로 평점 평균, 평점 개수 계산
ratings_mean_count = c_score.groupby('vod_id').agg({"ratings": ["mean", "count"]})
ratings_mean_count.columns = ["ratings_mean", "ratings_count"]

# 평점은 소수점 1자리에서 반올림
ratings_mean_count.ratings_mean = ratings_mean_count.ratings_mean.round(1)
ratings_mean_count

,ratings_mean,ratings_count
vod_id,,
0,3.0,1
1,2.0,1
3,2.8,5
5,1.0,1
6,2.8,6
...,...,...
4708,1.0,1
4710,2.0,1
4711,3.0,1


In [98]:
# 기존 추천 데이터에 평점 평균, 평점 개수 합쳐줍니다
recommendations = pd.merge(recommendations, ratings_mean_count, left_index=True, right_index=True, how="left")
# 거리순에 이은 평점, 평점수 순으로 정렬해줍니다.
recommendations.sort_values(["distance", "ratings_mean", "ratings_count"], ascending=[True, False, False], inplace=True)
recommendations

,vod_id,distance,ratings_mean,ratings_count
538,538,0.0,3.4,11.0
3838,3838,0.0,3.4,5.0
292,292,0.0,3.0,9.0
3124,3124,0.0,3.0,1.0
540,540,0.0,2.4,5.0
536,536,0.0,2.0,1.0
1046,1046,0.0,2.0,1.0
280,280,0.0,1.0,3.0
1387,1387,0.0,1.0,1.0
4519,4519,0.0,1.0,1.0


사용자별 추천

In [99]:
c_score

,subsr,rename,Category,vod_id,Affinity,ratings
0,59879000,나쁜엄마,TV프로그램,3403,0.405465,1.0
1,59879000,나의해방일지,TV프로그램,1859,0.405465,1.0
2,59879000,낭만닥터김사부2,TV프로그램,1168,0.405465,1.0
3,59879000,소방서옆경찰서,TV프로그램,2092,1.504077,3.0
4,59879000,슬기로운의사생활시즌2,TV프로그램,2891,0.405465,1.0
...,...,...,...,...,...,...
7700,67160000,심야괴담회,TV프로그램,143,0.637052,1.0
7701,67160000,콘크리트유토피아,영화,2630,0.659090,3.0
7702,67161000,스파이코드명포춘,영화,3824,1.705884,5.0
7703,67164000,미션임파서블:데드레코닝PARTONE,영화,2867,1.083268,1.0


In [100]:
# 67160000 계정이 본 vod_id
user = 67160000
watched = c_score[c_score['subsr'] == user]['vod_id']
watched
# vod_id = ratings_id[ratings_id['subsr'] == 59879000]['vod_id'][0]

7694    2058
7695    2866
7696    4018
7697    3523
7698    2057
7699    2147
7700     143
7701    2630
Name: vod_id, dtype: int64

In [101]:
# 한 계정이 여러 개 봤을 때 각각의 추천 모두 가져옴
# 계정 list
users = c_score['subsr'].unique()
users

array([59879000, 59882000, 59886000, 59890000, 59892000, 59893000,
       59895000, 59897000, 59899000, 59900000, 59901000, 59906000,
       59912000, 59917000, 59921000, 59922000, 59930000, 59933000,
       59937000, 60020000, 60026000, 60040000, 60050000, 60053000,
       60058000, 60067000, 60073000, 60082000, 60140000, 60145000,
       60146000, 60152000, 60169000, 60171000, 60182000, 60198000,
       60201000, 60207000, 60224000, 60252000, 60256000, 60260000,
       60265000, 60273000, 60302000, 60304000, 60306000, 60307000,
       60310000, 60320000, 60326000, 60333000, 60338000, 60381000,
       60411000, 60463000, 60466000, 60475000, 60492000, 60495000,
       60496000, 60506000, 60510000, 60513000, 60527000, 60533000,
       60543000, 60550000, 60561000, 60593000, 60599000, 60607000,
       60665000, 60675000, 60681000, 60684000, 60689000, 60696000,
       60707000, 60715000, 60726000, 60727000, 60735000, 60739000,
       60741000, 60780000, 60784000, 60790000, 60801000, 60813

In [102]:
# 한 계정이 본 vod_id 별로 추천 모두 가져옴
result = []
for subsr in users:
    recommendation_list = pd.DataFrame()
    watched = c_score[c_score['subsr'] == subsr]['vod_id']
    for i in watched:
        # 본 계정마다 추천 리스트 만듦
        vod = genres.iloc[i, :]
        distances, indices = nbrs.kneighbors([vod])
        recommendations = vod_list.loc[indices[0], ["vod_id"]]
        recommendations
        recommendations["distance"] = distances[0]
        # 기존 추천 데이터에 평점 평균, 평점 개수 합쳐줍니다
        recommendations = pd.merge(recommendations, ratings_mean_count, left_index=True, right_index=True, how="left")
        recommendation_list = pd.concat([recommendation_list, recommendations])
    
    # recommendation_list 에서 이미 시청한 것 중복제거
    recommendation_list = recommendation_list[~recommendation_list['vod_id'].isin(watched)]
    # 거리순에 이은 평점, 평점수 순으로 정렬해줍니다.
    recommendation_list.sort_values(["distance", "ratings_mean", "ratings_count"], ascending=[True, False, False], inplace=True)
    recommendation_list.drop_duplicates(inplace=True)
    # 상위 10개 vod_id 가져옴
    top_10 = recommendation_list['vod_id'].head(10)
    
    personal_top_10 = pd.DataFrame({'subsr': [subsr]})
    for idx, vod_id in enumerate(top_10, start=1):
        personal_top_10[f'vod_id_{idx}'] = vod_id
        
    result.append(personal_top_10)
    
result_df = pd.concat(result, ignore_index=True)
result_df

C:\Users\USER\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
C:\Users\USER\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
C:\Users\USER\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
C:\Users\USER\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
C:\Users\USER\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
C:\Users\USER\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have val

C:\Users\USER\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
C:\Users\USER\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
C:\Users\USER\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
C:\Users\USER\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
C:\Users\USER\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
C:\Users\USER\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have val

C:\Users\USER\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
C:\Users\USER\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
C:\Users\USER\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
C:\Users\USER\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
C:\Users\USER\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
C:\Users\USER\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have val

C:\Users\USER\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
C:\Users\USER\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
C:\Users\USER\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
C:\Users\USER\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
C:\Users\USER\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
C:\Users\USER\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have val

C:\Users\USER\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
C:\Users\USER\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
C:\Users\USER\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
C:\Users\USER\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
C:\Users\USER\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
C:\Users\USER\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have val

...

C:\Users\USER\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
C:\Users\USER\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
C:\Users\USER\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
C:\Users\USER\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
C:\Users\USER\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
C:\Users\USER\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have val

C:\Users\USER\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
C:\Users\USER\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
C:\Users\USER\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
C:\Users\USER\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
C:\Users\USER\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
C:\Users\USER\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have val

C:\Users\USER\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
C:\Users\USER\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
C:\Users\USER\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
C:\Users\USER\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
C:\Users\USER\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
C:\Users\USER\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have val

C:\Users\USER\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
C:\Users\USER\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
C:\Users\USER\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
C:\Users\USER\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
C:\Users\USER\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
C:\Users\USER\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have val

,subsr,vod_id_1,vod_id_2,vod_id_3,vod_id_4,vod_id_5,vod_id_6,vod_id_7,vod_id_8,vod_id_9,vod_id_10
0,59879000,1713,3773,3774,1135,1131,2977,1136,1712,1883,1133
1,59882000,3030,2799,4101,1412,3673,1176,4018,2866,4084,2655
2,59886000,1713,3467,3773,3774,637,1528,2057,3552,809,636
3,59890000,4101,4084,4089,1185,1281,1282,946,4173,4102,3993
4,59892000,1713,3467,3773,3774,538,3838,292,637,1528,3124
...,...,...,...,...,...,...,...,...,...,...,...
916,67149000,3467,538,3838,292,637,3124,1528,3552,809,540
917,67154000,538,3838,292,3124,540,536,1046,280,1387,4519
918,67160000,3467,4101,538,3838,292,637,1528,3124,3552,809
919,67161000,4018,4355,4376,4016,4019,2076,2095,4015,2705,4370


In [103]:
result_df.to_csv('genre_based_recommendation.csv', index=False)